In [1]:
import gym
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tensorboardX import SummaryWriter

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


class Model(nn.Module):
    def __init__(self, input_size, action_size):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 128) 
        )

        self.actor = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
        )

        self.critic = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        init_out = self.net(x)
        return self.actor(init_out), self.critic(init_out)

In [2]:
class Model(nn.Module):
    def __init__(self, input_size, action_size):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 24),
            nn.ReLU(),
            nn.Linear(24, action_size)
        )

    def forward(self, x):
        return self.net(x)

In [3]:
def train(b_states, b_actions, b_rewards, b_actor, b_critic, eps_steps, optimizer):
    
    #unpack batches for training
    for i in range(eps_steps):
        state = b_states[i]
        action = b_actions[i]
        #print('actions', action)
        reward = b_rewards[i]
        actor = b_actor[i]
        critic = b_critic[i]

        obs_v = torch.FloatTensor(state)
        rewards_v = torch.tensor(reward)
        critic = torch.FloatTensor(critic)
        actions_t = torch.tensor(action)

        loss_value_v = F.mse_loss(critic, rewards_v)

        log_prob_v = F.log_softmax(actor)
        adv_v = rewards_v - critic
        log_prob_actions_v = adv_v * log_prob_v[action]
        loss_policy_v = -log_prob_actions_v.mean()

        prob_v = F.softmax(actor)
        #entropy_loss_v = (prob_v * log_prob_v).sum().mean()
        #loss_v = (entropy_beta * entropy_loss_v + loss_value_v + loss_policy_v)
        loss_v = (loss_value_v + loss_policy_v)
        loss_v = torch.tensor(loss_v, requires_grad = True)
        #print('loss_v', loss_v)
        writer.add_scalar("loss", loss_v, i )

        loss_v.backward()

        optimizer.step()

    writer.close()

In [4]:
env = gym.make("Taxi-v3").env

model = Model(1, env.action_space.n)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, 
                             amsgrad=False)
optimizer.zero_grad()

In [5]:
state = env.reset()
done = False

nn_input = torch.LongTensor(1)
nn_input = nn_input.type(torch.LongTensor)
#nn_input[0] = state
action = model(nn_input)
        
new_state, reward, done, info = env.step(action)
        
state = new_state


RuntimeError: expected scalar type Long but found Float

writer = SummaryWriter()

for i in range(300):

    state = env.reset()
    #print('start pos', env.agent_pos)
    done = False
    b_states = []
    b_actions = []
    b_rewards = []
    b_new_state = []
    b_actor = []
    b_critic = []
    eps_reward = 0
    eps_steps = 0

    while not done:

        #create model input from flattened grid
        #nn_input = (tensor_convert(state))
        nn_input = torch.LongTensor(state).unsqueeze(dim=0)
        actor, critic = model(nn_input)
        b_actor.append(actor)
        b_critic.append(critic)

        #print('get new action')
        if random.random() < 0.05:
            action = env.action_space.sample()
        else:
            action = torch.argmax(actor)
        

        #print(F.softmax(actor), action)

        #run through step to book appointment
        new_state, reward, done, info = env.step(action)
        b_states.append(nn_input)
        b_actions.append(action)
        b_rewards.append(reward)
        b_new_state.append(new_state)
        eps_reward += reward
        eps_steps += 1

        #print('done', done)
        state = new_state

    #print("train with this", b_rewards, b_actions)
    #print('end of episode', eps_reward, eps_steps)
    #print(rewards_np[0], b_rewards[0], b_critic[0], tensor_rewards[0])
    train(b_states, b_actions, b_rewards, b_actor, b_critic, eps_steps, optimizer)
    writer.add_scalar("eps_reward", eps_reward, i )
    if i%100 == 0:
        print('rewards for episode', i, eps_reward)

writer.close()

In [ ]:
env.observation_space.n

In [ ]:
env.action_space.n

In [ ]:
state

In [ ]:
n=torch.tensor([1,2,3,4])
n

In [ ]:
nn_input = torch.LongTensor(1)
nn_input

In [ ]:
nn_input[0] = 4
type(nn_input)

In [ ]:
x = 448
y = torch.tensor(x).unsqueeze(-1)

In [ ]:
y.size()